### Import Required Libraries and Set Up Environment Variables

In [44]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')
print(NASA_API_KEY)

FhFvKGUQ7tNGtdldINWxXubqJ8yrhkF3lHgIFMZt


### CME Data

In [45]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
specifier = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url_CME = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [46]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url_CME)

In [47]:
# Convert the response variable to json and store it as a variable named cme_json
# Check if the request was successful

cme_json = cme_response.json()  # Convert the response to JSON
    

In [48]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [49]:
# Convert cme_json to a Pandas DataFrame 

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = pd.json_normalize(cme_json)

In [50]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])

In [51]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame

    # Iterate over each dictionary in the list
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows
for i in cme_df.index:
    # Get the current row's linkedEvents
    linkedEvents = cme_df.at[i, 'linkedEvents']
    
    # Ensure linkedEvents is a list before iterating
    if isinstance(linkedEvents, list):
        # Iterate over each dictionary in the list
        for event in linkedEvents:
            # Append a new dictionary to the expanded_rows list for each event
            expanded_rows.append({
                'activityID': cme_df.at[i, 'activityID'],
                'startTime': cme_df.at[i, 'startTime'],
                'linkedEvent': event  # Include the individual linked event
            })
        # Create a new DataFrame from the expanded rows
cme_expanded_df = pd.DataFrame(expanded_rows)

# Optional: Print the first few rows of the expanded DataFrame
print(cme_expanded_df.head())    

                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  


In [52]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging


def extract_activityID_from_dict(input_dict):
    try:
        # Attempt to return the 'activityID' from the dictionary
        return input_dict['activityID']
    except (KeyError, ValueError, TypeError) as e:
        # Handle the error gracefully and log or print it for debugging
        print(f"Error extracting activityID: {e}")  # Log the error for debugging
        return None  # Return None if there's an error


In [53]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
# Apply the function to create a new column for GST Activity IDs
cme_expanded_df['GST_ActivityID'] = cme_expanded_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))

# Drop rows with missing GST_ActivityID values
cme_expanded_df.dropna(subset=['GST_ActivityID'], inplace=True)

# Optionally, check the first few rows to verify the new column
print(cme_expanded_df[['linkedEvent', 'GST_ActivityID']].head())

                                     linkedEvent               GST_ActivityID
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001


In [54]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_expanded_df.dropna(subset=['GST_ActivityID'], inplace=True)

In [55]:
# print out the datatype of each column in this DataFrame:
print(cme_expanded_df.dtypes)

activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [56]:
# Convert the 'GST_ActivityID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_CME and activityID to cmeID

# Drop linkedEvents

# Verify that all steps were executed correctly
# Convert the 'GST_ActivityID' column to string format
cme_expanded_df['GST_ActivityID'] = cme_expanded_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format
cme_expanded_df['startTime'] = pd.to_datetime(cme_expanded_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme_expanded_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop the 'linkedEvents' column
cme_expanded_df.drop(columns=['linkedEvent'], inplace=True)

# Verify that all steps were executed correctly
print(cme_expanded_df.dtypes)  # Check the data types of the columns
print(cme_expanded_df.head())   # Preview the first few rows to confirm changes

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object
                         cmeID             startTime_CME  \
0  2013-05-01T03:12:00-CME-001 2013-05-01 03:12:00+00:00   
1  2013-05-03T22:36:00-CME-001 2013-05-03 22:36:00+00:00   
2  2013-05-09T19:29:00-CME-001 2013-05-09 19:29:00+00:00   
3  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   
4  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   

                GST_ActivityID  
0  2013-05-04T04:52:00-IPS-001  
1  2013-05-07T04:37:00-IPS-001  
2  2013-05-12T23:30:00-IPS-001  
3  2013-05-13T01:53:00-FLR-001  
4  2013-05-13T04:12:00-SEP-001  


In [57]:
# Filter the DataFrame to keep only rows where 'GST_ActivityID' contains 'GST'
cme_filtered_df = cme_expanded_df[cme_expanded_df['GST_ActivityID'].str.contains('GST', na=False)]

# Optionally, print the shape of the filtered DataFrame to see how many rows are left
print("Number of rows after filtering for GSTs:", cme_filtered_df.shape[0])

# Preview the first few rows of the filtered DataFrame
print(cme_filtered_df.head())

Number of rows after filtering for GSTs: 61
                           cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:00-GST-001  
48   2013-10-02T03:00:00-GST-001  
90   2013-12-08T00:00:00-GST-001  
148  2014-02-19T03:00:00-GST-001  
151  2014-02-20T03:00:00-GST-001  


### GST Data

In [58]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_GST = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
print("Query URL for GST:", query_url_GST)

Query URL for GST: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=FhFvKGUQ7tNGtdldINWxXubqJ8yrhkF3lHgIFMZt


In [59]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)

In [60]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
gst_json = gst_response.json()  # Convert the response to JSON
print(json.dumps(gst_json[0], indent=4))  # Print the first result with indentation

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [62]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.json_normalize(gst_json)
print(gst_df.head())
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['activityID', 'startTime', 'linkedEvents']]

                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   

                                          allKpIndex  \
0  [{'observedTime': '2013-06-01T01:00Z', 'kpInde...   
1  [{'observedTime': '2013-06-07T03:00Z', 'kpInde...   
2  [{'observedTime': '2013-06-29T03:00Z', 'kpInde...   
3  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   
4  [{'observedTime': '2013-12-08T03:00Z', 'kpInde...   

                                                link  \
0  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
1  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
2  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
3  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   
4  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...   

       

KeyError: "['activityID'] not in index"

In [63]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])

In [64]:
# Explode the 'linkedEvents' column so that each event gets its own row
gst_exploded_df = gst_df.explode('linkedEvents')

# Reset the index of the exploded DataFrame
gst_exploded_df.reset_index(drop=True, inplace=True)
# Drop any rows with missing values in 'linkedEvents' (if any exist after explosion)
gst_exploded_df.dropna(subset=['linkedEvents'], inplace=True)

In [65]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_exploded_df['CME_ActivityID'] = gst_exploded_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_exploded_df.dropna(subset=['CME_ActivityID'], inplace=True)


In [66]:
# Convert the 'CME_ActivityID' column to string format 
gst_exploded_df['CME_ActivityID'] = gst_exploded_df['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_exploded_df['gstID'] = gst_exploded_df['gstID'].astype(str)
# Convert startTime to datetime format  
gst_exploded_df['startTime'] = pd.to_datetime(gst_exploded_df['startTime'])
# Rename startTime to startTime_GST 
gst_exploded_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)
# Drop linkedEvents
gst_exploded_df.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
print(gst_exploded_df.dtypes)  # Check the data types of the columns
print(gst_exploded_df.head())   # Preview the first few rows to confirm changes

gstID                          object
startTime_GST     datetime64[ns, UTC]
allKpIndex                     object
link                           object
submissionTime                 object
versionId                       int64
CME_ActivityID                 object
dtype: object
                         gstID             startTime_GST  \
0  2013-06-01T01:00:00-GST-001 2013-06-01 01:00:00+00:00   
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
3  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
4  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   

                                          allKpIndex  \
0  [{'observedTime': '2013-06-01T01:00Z', 'kpInde...   
1  [{'observedTime': '2013-06-07T03:00Z', 'kpInde...   
2  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   
3  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   
4  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   

                      

In [67]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
# Filter the DataFrame to keep only rows where 'CME_ActivityID' contains 'CME'
gst_filtered_df = gst_exploded_df[gst_exploded_df['CME_ActivityID'].str.contains('CME', na=False)]

# Optionally, print the shape of the filtered DataFrame to see how many rows are left
print("Number of rows after filtering for CMEs:", gst_filtered_df.shape[0])

# Preview the first few rows of the filtered DataFrame
print(gst_filtered_df.head())

Number of rows after filtering for CMEs: 61
                         gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                                          allKpIndex  \
1  [{'observedTime': '2013-06-07T03:00Z', 'kpInde...   
2  [{'observedTime': '2013-10-02T06:00Z', 'kpInde...   
5  [{'observedTime': '2013-12-08T03:00Z', 'kpInde...   
7  [{'observedTime': '2014-02-19T06:00Z', 'kpInde...   
9  [{'observedTime': '2014-02-20T06:00Z', 'kpInde...   

                                                link     submissionTime  \
1  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...  2013-07-15T19:41Z   
2  https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...  2013-10-02T13:23Z   
5  https://webtools.ccmc.gsfc.nas

### Merge both datatsets

In [68]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    gst_filtered_df,
    cme_filtered_df,
    left_on='CME_ActivityID',  # Key from the GST DataFrame
    right_on='GST_ActivityID',  # Key from the CME DataFrame
    how='inner'  # Change this to 'left', 'right', or 'outer' if needed
)


In [69]:
# Verify that the new DataFrame has the same number of rows as cme and gst
cme_row_count = cme_filtered_df.shape[0]
gst_row_count = gst_filtered_df.shape[0]
merged_row_count = merged_df.shape[0]

# Print the results
print("Number of rows in CME DataFrame:", cme_row_count)
print("Number of rows in GST DataFrame:", gst_row_count)
print("Number of rows in merged DataFrame:", merged_row_count)


if merged_row_count == min(cme_row_count, gst_row_count):
    print("The merged DataFrame has the expected number of rows.")
else:
    print("The merged DataFrame does NOT have the expected number of rows.")

Number of rows in CME DataFrame: 61
Number of rows in GST DataFrame: 61
Number of rows in merged DataFrame: 0
The merged DataFrame does NOT have the expected number of rows.


### Computing the time it takes for a CME to cause a GST

In [71]:
#compute the time diff between startTime_GST and startTime_CME by creating a new colu# Cmn called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

In [73]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 

time_diff_stats = merged_df['timeDiff'].describe()

# Extract the mean and median
mean_time_diff = time_diff_stats['mean']
median_time_diff = time_diff_stats['50%']  # The 50th percentile is the median

# Print the results
print("Mean time difference (CME to GST):", mean_time_diff)
print("Median time difference (CME to GST):", median_time_diff)

Mean time difference (CME to GST): NaT
Median time difference (CME to GST): NaT


### Exporting data in csv format

In [74]:
# Export data to CSV without the index
merged_df.to_csv('cme_gst_data.csv', index=False)